In [ ]:
import medmnist
from medmnist import INFO, Evaluator
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import numpy as np
import sys

data_flag = 'chestmnist' 
info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])
train_data = DataClass(split='train', download=True)
test_data = DataClass(split='test', download=True)
val_data = DataClass(split='val', download=True)

X_train, y_train = train_data.imgs, train_data.labels
X_val, y_val = val_data.imgs, val_data.labels
X_test, y_test = test_data.imgs, test_data.labels

# Normalize images to [0, 1]
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

In [ ]:
# Convert one-hot encoded labels to class indices
y_train = np.argmax(y_train, axis=1)
y_val = np.argmax(y_val, axis=1)
y_test = np.argmax(y_test, axis=1)

In [ ]:
# Convert all classes to 1 or 0
def convert2Binary(npArr):
    for i in range(len(npArr)):
        if(npArr[i] != 0):
            npArr[i] = 1
    return npArr
y_train = convert2Binary(y_train)
y_val = convert2Binary(y_val)
y_test = convert2Binary(y_test)

In [ ]:
binary_classes = [0, 1]

print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, n_channels)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32
)

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")

y_pred = (model.predict(X_test) > 0.5).astype('int32')

from sklearn.metrics import classification_report, confusion_matrix
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()